# Yelp Parse

In [85]:
# Import needed package
import requests
import json
import csv
import re
import pandas as pd
import numpy as np
# Import BeautifulSoup to parse HTML
from bs4 import BeautifulSoup

In [64]:
african = pd.read_csv('Export/african.csv')
african_alias = african['alias']
for i in african_alias:
    print(i)

salare-seattle
safari-njema-restaurant-seattle
blackstar-kebabs-seattle
on-safari-café-seattle
malaaya-restaurant-seattle
mama-sambusa-s-small-kitchen-seattle
bahati-restaurant-seattle


## Create user agents

In [382]:
def get_random_ua():
    ua_list = pd.read_csv('Data/ua_file.csv',sep="\n",header=None)
    ua_list = np.array(ua_list)
    indices = np.arange(len(ua_list))
    rnd_indices = np.random.choice(indices, size=1)
    return ua_list[rnd_indices][0][0]

In [381]:
test = pd.read_csv('Data/ua_file.csv',sep="\n",header=None)
test = np.array(test)
indices = np.arange(len(test))
rnd_indices = np.random.choice(indices, size=1)
test[rnd_indices][0][0]

'Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24'

## Scrape the website by parsing html

In [50]:
# Request webpage information by parsing html
count = 0
african_alias = african['alias']
african_reviews = []

for restaurant in african_alias:
    base_url = "https://www.yelp.com/biz/"
    r = requests.get(base_url + restaurant)
    african_reviews.append({"alias":""}) 
    african_reviews[count]['alias']=restaurant
    soup = BeautifulSoup(r.text, 'html.parser')
    #reviews
    results = soup.find_all(attrs={'class': re.compile(r"^lemon--span__373c0__3997G$"),'lang':'en'})
    reviews = []
    for result in results:
        reviews.append(result.text)
    african_reviews[count]['reviews']=reviews
    count +=1

# Make sure that has scraped at least 100 recipes
#print(scrape_count,recipe_num,"succeed")
african_reviews

[{'alias': 'salare-seattle',
  'reviews': ["Met a friend for Sunday brunch, and she chose Salare. What a lovely spot! From the moment you walk in, it has a very 'brunch' feel (if that makes sense). Loved the clean and fresh decor as well. For brunch we started with the french toast with jam and creme fraiche - so good! The french toast was melt in your mouth (with the perfect amount of crispiness). \xa0I also had the quiche ( again - the perfect crust - I do love me some crispiness ;)). My friend had the mushroom ragou which was very hearty. \xa0Parking can be difficult - it's in a residential-ly area so some tight streets and limited street parking. But I would love to come back for dinner - Ravenna is such a cozy little neighborhood!",
   "We came here to celebrate the birthday boy's big day and had a great dinner made possible by the friendly staff, attention to plating details, and unique flavor combinations. Salare is an eclectic restaurant that features America's diverse culture 

## Scrape the website by looking into json file

In [185]:
# Request webpage information by json
count = 0
african_alias = african['alias']
african_reviews = pd.DataFrame()

for restaurant in african_alias:
    base_url = "https://www.yelp.com/biz/"
    r = requests.get(base_url + restaurant)
    soup = BeautifulSoup(r.text, 'html.parser')
    # find how many pages
    page = soup.find_all('span', 
                        attrs={'class': re.compile(r"^lemon--span__373c0__3997G text__373c0__2pB8f text-color--inherit__373c0__w_15m text-align--left__373c0__2pnx_$")})
    page = int(page[1].text.split(" ")[3])
    for page_count in range(page):
        page_count = page_count*20
    reviews = soup.find_all('script', text=re.compile('aggregateRating'))
    reviews = json.loads(reviews[0].text)
    temp = {}
    for i in range(len(reviews['review'])):
        temp['restaurant']=restaurant
        temp['rating'] = reviews['review'][i]['reviewRating']['ratingValue']
        temp['datePublished'] = reviews['review'][i]['datePublished']
        temp['text'] = reviews['review'][i]['description']
        temp['author'] = reviews['review'][i]['author']
        temp_df = pd.DataFrame([temp])
        african_reviews = pd.concat([african_reviews,temp_df],sort=False)
    count +=1

african_reviews

,restaurant,rating,datePublished,text,author
0,salare-seattle,4,2020-03-01,"Met a friend for Sunday brunch, and she chose ...",Cristina P.
0,salare-seattle,4,2020-02-22,We came here to celebrate the birthday boy's b...,Courtney L.
0,salare-seattle,4,2020-02-12,A nice and cozy restaurant that's on a more hi...,Maggie S.
0,salare-seattle,4,2020-01-13,I came here with my family to give it a try an...,Sebastian T.
0,salare-seattle,5,2020-03-02,"Lovely experience, they were able to take the ...",Alyssa P.
...,...,...,...,...,...
0,bahati-restaurant-seattle,5,2019-03-31,I really love this place. They have high quali...,Sarah S.
0,bahati-restaurant-seattle,5,2016-10-16,Seriously bomb good food. The spice is awesom...,Staci M.
0,bahati-restaurant-seattle,3,2015-09-08,Not going to give a negative rating because I ...,Kellen C.
0,bahati-restaurant-seattle,5,2016-12-31,So amazing. Ordered from this place on uber ea...,riss b.


In [ ]:
# Request webpage information by json
african_alias = african['alias']
all_reviews = pd.DataFrame()

for restaurant in african_alias:
    user_agent = get_random_ua()
    headers = {'user-agent': user_agent}
    base_url = "https://www.yelp.com/biz/"
    r = requests.get(base_url + restaurant, headers=headers)
    soup = BeautifulSoup(r.text, 'html.parser')
    # find how many pages
    page = soup.find_all('span', 
                        attrs={'class': re.compile(r"^lemon--span__373c0__3997G text__373c0__2pB8f text-color--inherit__373c0__w_15m text-align--left__373c0__2pnx_$")})
    page = int(page[1].text.split(" ")[3])
    
    restaurant_reviews = pd.DataFrame()
    
    # parse through pages
    for page_count in range(page):
        page_reviews = pd.DataFrame()
        page_count = str(page_count*20)
        r = requests.get(base_url + restaurant+"?start="+page_count)
        soup = BeautifulSoup(r.text, 'html.parser')
        
        # reviews
        reviews = soup.find_all('script', text=re.compile('aggregateRating'))
        reviews = json.loads(reviews[0].text)
        temp = {}
        for i in range(len(reviews['review'])):
            temp['restaurant']=restaurant
            temp['rating'] = reviews['review'][i]['reviewRating']['ratingValue']
            temp['datePublished'] = reviews['review'][i]['datePublished']
            temp['text'] = reviews['review'][i]['description']
            temp['author'] = reviews['review'][i]['author']
            temp_df = pd.DataFrame([temp])
            page_reviews = pd.concat([page_reviews,temp_df],sort=False)


        # location of reveiwer
        places = soup.find_all('span', attrs={'class': re.compile(r"^lemon--span__373c0__3997G text__373c0__2pB8f text-color--normal__373c0__K_MKN text-align--left__373c0__2pnx_ text-weight--bold__373c0__3HYJa text-size--small__373c0__3SGMi$")})
        place_temp = []
        for place in places:
            place_temp.append(place.text.split(", ")[-2:])
        place_temp = pd.DataFrame(place_temp,columns=['City','State'])
        
        # number of friends and reviews of reviewer
        reviewers = soup.find_all('span', attrs={'class': re.compile(r"^lemon--span__373c0__3997G text__373c0__2pB8f text-color--normal__373c0__K_MKN text-align--left__373c0__2pnx_ text-size--small__373c0__3SGMi$")})
        friend_temp = []
        review_temp = []
        for data in reviewers:
            data_temp = data.text.split(" ")
            if data_temp[1]=='friends':
                friend_temp.append(data_temp[0])
            elif data_temp[1]=='reviews':
                review_temp.append(data_temp[0])
        friend_temp = pd.DataFrame(friend_temp,columns=['Friends'])
        review_temp = pd.DataFrame(review_temp,columns=['Num_reviews'])
        
        # combine all the data
        page_reviews.reset_index(drop=True, inplace=True)
        page_reviews = pd.concat([page_reviews,place_temp,friend_temp,review_temp],axis=1,sort=False)
        restaurant_reviews = pd.concat([restaurant_reviews,page_reviews],sort=False)
    all_reviews = pd.concat([all_reviews,restaurant_reviews],sort=False)

In [334]:
all_reviews.to_csv('Export/african_reviews.csv')

In [339]:
all_reviews[all_reviews['Friends'].isna()]

,restaurant,rating,datePublished,text,author,City,State,Friends,Num_reviews
19,salare-seattle,2,2018-09-17,"All 4 dishes we ordered (Lamb tartare, Potato ...",Andy O.,Federal Way,WA,NaN,10
18,salare-seattle,3,2018-12-09,Didn't love it. Had a reservation for my son's...,Christie R.,Seattle,WA,NaN,3
19,salare-seattle,2,2018-04-13,After hearing about Salare and Junebaby from w...,Cassandra K.,Costa Mesa,CA,NaN,307
19,salare-seattle,5,2018-10-13,This place gets a high five! Hip ambiance and ...,Mischa A.,Brooklyn,NY,NaN,11
0,salare-seattle,2,2016-11-26,Bummer\n\nI really want to like this place but...,Jennifer B.,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2,bahati-restaurant-seattle,5,2019-03-31,I really love this place. They have high quali...,Sarah S.,NaN,NaN,NaN,NaN
3,bahati-restaurant-seattle,5,2016-10-16,Seriously bomb good food. The spice is awesom...,Staci M.,NaN,NaN,NaN,NaN
4,bahati-restaurant-seattle,3,2015-09-08,Not going to give a negative rating because I ...,Kellen C.,NaN,NaN,NaN,NaN
5,bahati-restaurant-seattle,5,2016-12-31,So amazing. Ordered from this place on uber ea...,riss b.,NaN,NaN,NaN,NaN


In [340]:
all_reviews.restaurant.value_counts()

salare-seattle                          385
safari-njema-restaurant-seattle         200
blackstar-kebabs-seattle                 30
on-safari-café-seattle                   10
bahati-restaurant-seattle                 7
mama-sambusa-s-small-kitchen-seattle      2
malaaya-restaurant-seattle                2
Name: restaurant, dtype: int64

In [235]:
african_reviews.to_csv('Export/african_reviews.csv')

In [387]:
user_agent = get_random_ua()
headers = {'user-agent': user_agent}
restaurant = 'salare-seattle'
base_url = "https://www.yelp.com/biz/"
r = requests.get(base_url + restaurant,headers=headers)
#soup = BeautifulSoup(r.text, 'html.parser')
#num_review = soup.find_all('span', attrs={'class': re.compile(r"^lemon--span__373c0__3997G text__373c0__2pB8f text-color--normal__373c0__K_MKN text-align--left__373c0__2pnx_ text-size--small__373c0__3SGMi$")})
r.status_code


Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.62 Safari/537.36


503

In [299]:
restaurant = 'salare-seattle'
base_url = "https://www.yelp.com/biz/"
r = requests.get(base_url + restaurant)
soup = BeautifulSoup(r.text, 'html.parser')
reviewers = soup.find_all('span', 
                        attrs={'class': re.compile(r"^lemon--span__373c0__3997G text__373c0__2pB8f text-color--normal__373c0__K_MKN text-align--left__373c0__2pnx_ text-size--small__373c0__3SGMi$")})
friend_temp = []
review_temp = []
for data in reviewers:
    data_temp = data.text.split(" ")
    if data_temp[1]=='friends':
        friend_temp.append(data_temp[0])
    elif data_temp[1]=='reviews':
        review_temp.append(data_temp[0])

AttributeError: 'list' object has no attribute 'shape'

In [292]:
test = pd.DataFrame([1,2,3])
test2 = pd.DataFrame([4,5,6])
test3 = pd.DataFrame([7,2,3])
pd.concat([test,test2,test3],axis=1,sort=False)

,0,0,0
0,1,4,7
1,2,5,2
2,3,6,3


In [290]:
friend_temp = []
review_temp = []
for friend in friends:
    test_temp = friend.text.split(" ")
    if test_temp[1]=='friends':
        friend_temp.append(test_temp[0])
    elif test_temp[1]=='reviews':
        review_temp.append(test_temp[0])
pd.DataFrame(friend_temp,columns=['Friends'])

['134', 'friends']
['489', 'reviews']
['178', 'photos']
['299', 'friends']
['24', 'reviews']
['55', 'photos']
['35', 'friends']
['24', 'reviews']
['78', 'photos']
['345', 'friends']
['323', 'reviews']
['1168', 'photos']
['61', 'friends']
['9', 'reviews']
['1', 'photo']
['0', 'friends']
['103', 'reviews']
['10', 'photos']
['77', 'friends']
['55', 'reviews']
['16', 'photos']
['196', 'friends']
['535', 'reviews']
['294', 'photos']
['68', 'friends']
['128', 'reviews']
['277', 'photos']
['3', 'friends']
['14', 'reviews']
['14', 'photos']
['341', 'friends']
['2230', 'reviews']
['3360', 'photos']
['54', 'friends']
['86', 'reviews']
['53', 'photos']
['187', 'friends']
['452', 'reviews']
['2503', 'photos']
['228', 'friends']
['18', 'reviews']
['87', 'photos']
['211', 'friends']
['396', 'reviews']
['2984', 'photos']
['0', 'friends']
['15', 'reviews']
['4', 'photos']
['4', 'friends']
['38', 'reviews']
['10', 'photos']
['283', 'friends']
['136', 'reviews']
['258', 'photos']
['151', 'friends']
['37

,Friends
0,134
1,299
2,35
3,345
4,61
5,0
6,77
7,196
8,68
9,3


In [276]:
test = [[1,'a'],[2,'b'],[3,'a'],[4,'b']]
pd.DataFrame(test)

,0,1
0,1,a
1,2,b
2,3,a
3,4,b


In [255]:
place_temp = []
for place in places:
    place_temp.append(place.text.split(", ")[-2:])
pd.DataFrame(place_temp,columns=['City','State'])
axis=1

[['Bellevue', 'WA'],
 ['Seattle', 'WA'],
 ['Seattle', 'WA'],
 ['Seattle', 'WA'],
 ['Renton', 'WA'],
 ['Tacoma', 'WA'],
 ['Bothell', 'WA'],
 ['Alameda', 'CA'],
 ['Arlington', 'VA'],
 ['Seattle', 'WA'],
 ['Woodinville', 'WA'],
 ['Bellevue', 'WA'],
 ['Seattle', 'WA'],
 ['Seattle', 'WA'],
 ['Seattle', 'WA'],
 ['Bothell', 'WA'],
 ['Los Angeles', 'CA'],
 ['Seattle', 'WA'],
 ['Renton', 'WA'],
 ['San Antonio', 'TX']]

In [258]:
pd.DataFrame(place_temp,columns=['City','State'])

,City,State
0,Bellevue,WA
1,Seattle,WA
2,Seattle,WA
3,Seattle,WA
4,Renton,WA
5,Tacoma,WA
6,Bothell,WA
7,Alameda,CA
8,Arlington,VA
9,Seattle,WA


In [217]:
results.find_next("span")

<span class="lemon--span__373c0__3997G text__373c0__2pB8f text-color--inherit__373c0__w_15m text-align--left__373c0__2pnx_ text-weight--bold__373c0__3HYJa">Yelp Sort</span>

In [230]:
page = soup.find_all('span', 
                        attrs={'class': re.compile(r"^lemon--span__373c0__3997G text__373c0__2pB8f text-color--inherit__373c0__w_15m text-align--left__373c0__2pnx_$")})
page[1].text.split(" ")[3]

'10'